In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mltools as ml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,confusion_matrix,roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals import joblib


In [87]:
#train_data = np.genfromtxt('data/transformed_train.txt',delimiter = None)
#train_Y = np.genfromtxt('data/train_Y.txt',delimiter = None)
from imblearn.combine import SMOTETomek

sc1 = StandardScaler()
X = np.genfromtxt("data/X_train.txt",delimiter=None)
Y = np.genfromtxt("data/Y_train.txt",delimiter=None)
train_data = X
train_Y = Y
smotetomek = SMOTETomek(random_state = 0)
sc1.fit(train_data)
train_data = sc1.transform(train_data)
train_data,train_Y = smotetomek.fit_sample(train_data,train_Y)

In [88]:
#Adaboost
alpha = np.zeros(shape=(train_data.shape[1],1))
weights_models = np.ones(shape=(train_data.shape[0],))
#predictions_val = np.zeros(shape=(val_data.shape[0],val_data.shape[1]))
for i in range(0,train_data.shape[1]):
    model = DecisionTreeClassifier()
    model.fit(train_data[:,i].reshape(-1,1),train_Y,np.atleast_1d(weights_models))
    joblib.dump(model,'data/decision_model_'+str(i)+'.txt')
    predictions = model.predict(train_data[:,i].reshape(-1,1))
#    predictions_val[:,i] = model.predict(val_data[:,i].reshape(-1,1))
    error = np.dot(weights_models,train_Y!=predictions)
    alpha[i] = 0.5*np.log(np.absolute((1-error)/error))
    weights_models *= np.exp(-alpha[i]*train_Y*predictions)
    weights_models /= weights_models.sum()

In [92]:
test_data = np.genfromtxt("data/X_test.txt",delimiter = None)
test_data = sc1.transform(test_data)
#test_data = np.genfromtxt('data/transformed_test.txt',delimiter = None)
test_predictions = np.zeros(shape=(test_data.shape[0],test_data.shape[1]))
for i in range(0,14):
    model = joblib.load('data/decision_model_'+str(i)+'.txt')
    test_predictions[:,i] = model.predict_proba(test_data[:,i].reshape(-1,1))[:,1]

final_test_predictions = np.dot(test_predictions,alpha)
final_test_predictions /= sum(alpha)

In [93]:
print(np.count_nonzero(final_test_predictions>0.5))

10414


In [94]:
Yte = np.vstack((np.arange(test_data.shape[0]), final_test_predictions.T)).T
np.savetxt('data/Y_submit.txt', Yte, '%d, %.2f', header='ID,Prob1', comments='', delimiter=',')